In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

base_url = "https://chothuephongtro.me"
headers = {"User-Agent": "Mozilla/5.0"}

records = []

#Lấy dữ liệu qua các bài đăng
for page_num in range(1,100):
    url = f"{base_url}/da-nang.html?page={page_num}"
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    for post in soup.find_all("article", class_="post-item"):

        price = post.find("span", class_="post-price").get_text(strip=True) if post.find("span", class_="post-price") else None
        area = post.find("span", class_="acreage").get_text(strip=True) if post.find("span", class_="acreage") else None
        date = post.find("span", class_="date").get_text(strip=True) if post.find("span", class_="date") else None
        location = post.find("span", class_="location").get_text(strip=True) if post.find("span", class_="location") else None

        #Truy cập vào từng bài đăng để lấy thêm thông tin chi tiét
        link_tag = post.find("a")
        detail_url = None
        #Kiểm tra link_tag có tồn tại và có chứa href không
        if link_tag and link_tag.get("href"):
            detail_url = base_url + link_tag["href"] if link_tag["href"].startswith("/") else link_tag["href"]

        if detail_url:
            try:
                detail_res = requests.get(detail_url, headers=headers)
                detail_soup = BeautifulSoup(detail_res.text, "html.parser")
                cond_tag = detail_soup.find("div", class_="section-content")
                details = None
                if cond_tag:
                  paragraphs = cond_tag.find_all("p")
                  all_details = [p.get_text(strip=True) for p in paragraphs]
                  washer      = "1" if any("máy giặt" in d or "full" in d.lower() or "đầy đủ" in d.lower() for d in all_details) else "0"
                  aircon      = "1" if any("điều hòa" in d or "full" in d.lower() or "đầy đủ" in d.lower() for d in all_details) else "0"
                  fridge      = "1" if any("tủ lạnh" in d or "full" in d.lower() or "đầy đủ" in d.lower() for d in all_details) else "0"
                  wifi        = "1" if any("wifi" in d or "full" in d.lower() or "đầy đủ" in d.lower() for d in all_details) else "0"
            except Exception as e:
                print(f"Lỗi khi crawl {detail_url}: {e}")

        #Chuyển thành dictionary để tạo bảng bằng pandas
        record = {
            "Price": price,
            "Area": area,
            "Date": date,
            "Location": location,
            "Detail URL": detail_url,
            "fridge": fridge,
            "washer": washer,
            "air condition" : aircon,
            "wifi" : wifi
        }
        records.append(record)
        #Cài khoảng nghỉ để ko bị web chặn và dính capcha
        time.sleep(0.5)


df = pd.DataFrame(records)
df

,Price,Area,Date,Location,Detail URL,fridge,washer,air condition,wifi
0,1.8triệu/tháng,18m2,1 tuần trước,"Hòa Vang, Đà Nẵng",https://chothuephongtro.me/cho-thue-phong-tro-...,1,1,1,1
1,6.9triệu/tháng,50m2,2 tháng trước,"Cẩm Lệ, Đà Nẵng",https://chothuephongtro.me/cho-thue-can-ho-1k1...,1,1,1,1
2,5triệu/tháng,35m2,2 tháng trước,"Hải Châu, Đà Nẵng",https://chothuephongtro.me/nha-cho-thue-hem-ph...,1,1,1,1
3,2.5triệu/tháng,23m2,4 tháng trước,"Hải Châu, Đà Nẵng",https://chothuephongtro.me/da-nang/hai-chau/15...,0,0,0,1
4,1triệu/tháng,20m2,4 tháng trước,"Hải Châu, Đà Nẵng",https://chothuephongtro.me/da-nang/hai-chau/12...,0,0,0,0
...,...,...,...,...,...,...,...,...,...
409,2triệu/tháng,25m2,4 năm trước,"Hải Châu, Đà Nẵng",https://chothuephongtro.me/da-nang/hai-chau/12...,0,0,0,0
410,2.3triệu/tháng,25m2,4 năm trước,"Ngũ Hành Sơn, Đà Nẵng",https://chothuephongtro.me/da-nang/ngu-hanh-so...,0,0,0,0
411,Thỏa thuận,35m2,4 năm trước,"Thanh Khê, Đà Nẵng",https://chothuephongtro.me/da-nang/thanh-khe/1...,1,1,1,1
412,1.7triệu/tháng,28m2,4 năm trước,"Hải Châu, Đà Nẵng",https://chothuephongtro.me/da-nang/hai-chau/12...,1,1,1,1


In [6]:
from datetime import datetime, timedelta
import re
#làm sạch cột Price : 6.9triệu/tháng50m2' → 6.9
def clean_price(price):
    if pd.isna(price):
        return None
    match = re.search(r'(\d+(\.\d+)?)', str(price))
    return float(match.group(1))* 1_000_000 if match else None

df['Price'] = df['Price'].apply(clean_price)

#làm sạch Area
def clean_area(area):
    if pd.isna(area):
        return None
    # Tìm số có thể có phần thập phân
    match = re.search(r"(\d+(\.\d+)?)", str(area))
    return float(match.group(1)) if match else None

df["Area"] = df["Area"].apply(clean_area)


# Làm sạch cột Date
def convert_date(text):
    today = datetime.today()
    if pd.isna(text):
        return None
    text = str(text).strip().lower()

    # xử lí năm
    y = re.match(r'(\d+)\s*năm\s*trước', text)
    if y:
        years_ago = int(y.group(1))
        new_date = today - pd.DateOffset(years=years_ago)
        return new_date.strftime('%d/%m/%Y')

    # xử lí tháng
    m = re.match(r'(\d+)\s*tháng\s*trước', text)
    if m:
        months_ago = int(m.group(1))
        new_date = today - pd.DateOffset(months=months_ago)
        return new_date.strftime('%d/%m/%Y')

    # xử lý tuần
    w = re.match(r'(\d+)\s*tuần\s*trước', text)
    if w:
        weeks_ago = int(w.group(1))
        new_date = today - timedelta(weeks=weeks_ago)
        return new_date.strftime('%d/%m/%Y')

    #Xử lí ngày
    d = re.match(r'(\d+)\s*ngày\s*trước', text)
    if d:
        days_ago = int(d.group(1))
        new_date = today - timedelta(days=days_ago)
        return new_date.strftime('%d/%m/%Y')

    # Nếu text đã ở dạng dd/mm/yyyy, giữ nguyên
    if re.match(r'\d{2}/\d{2}/\d{4}', text):
        return text

    # Nếu không khớp, giữ nguyên
    return text

df['Date'] = df['Date'].apply(convert_date)
# Chuyển '07/09/2025' -> '2025-09-07' để MySQL hiểu được
df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y", errors="coerce").dt.strftime("%Y-%m-%d")
df


,Price,Area,Date,Location,Detail URL,fridge,washer,air condition,wifi
0,1800000.0,18.0,2025-10-23,"Hòa Vang, Đà Nẵng",https://chothuephongtro.me/cho-thue-phong-tro-...,1,1,1,1
1,6900000.0,50.0,2025-08-30,"Cẩm Lệ, Đà Nẵng",https://chothuephongtro.me/cho-thue-can-ho-1k1...,1,1,1,1
2,5000000.0,35.0,2025-08-30,"Hải Châu, Đà Nẵng",https://chothuephongtro.me/nha-cho-thue-hem-ph...,1,1,1,1
3,2500000.0,23.0,2025-06-30,"Hải Châu, Đà Nẵng",https://chothuephongtro.me/da-nang/hai-chau/15...,0,0,0,1
4,1000000.0,20.0,2025-06-30,"Hải Châu, Đà Nẵng",https://chothuephongtro.me/da-nang/hai-chau/12...,0,0,0,0
...,...,...,...,...,...,...,...,...,...
409,2000000.0,25.0,2021-10-30,"Hải Châu, Đà Nẵng",https://chothuephongtro.me/da-nang/hai-chau/12...,0,0,0,0
410,2300000.0,25.0,2021-10-30,"Ngũ Hành Sơn, Đà Nẵng",https://chothuephongtro.me/da-nang/ngu-hanh-so...,0,0,0,0
411,NaN,35.0,2021-10-30,"Thanh Khê, Đà Nẵng",https://chothuephongtro.me/da-nang/thanh-khe/1...,1,1,1,1
412,1700000.0,28.0,2021-10-30,"Hải Châu, Đà Nẵng",https://chothuephongtro.me/da-nang/hai-chau/12...,1,1,1,1


In [ ]:
#df.dropna(how='all', inplace=True)

# Loại bỏ dòng có ít nhất 1 giá trị trống
#df.dropna(how='any', inplace=True)

In [4]:
print(df.isna().sum())

Price            0
Area             0
Date             0
Location         0
Detail URL       0
fridge           0
washer           0
air condition    0
wifi             0
dtype: int64


In [18]:
import mysql.connector
import pandas as pd
conn = mysql.connector.connect(
    host="localhost",       
    user="root",             
    password="Minhthinh@2k5",
    database="DB_STORAGE2"   
)
cursor = conn.cursor()
print("Kết nối MySQL thành công!")


Kết nối MySQL thành công!


In [20]:
import numpy as np

df = df.replace({np.nan: None, "nan": None, "NaN": None})

In [21]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS houses (
    id INT AUTO_INCREMENT PRIMARY KEY,
    location VARCHAR(255),
    price VARCHAR(50),
    area VARCHAR(50),
    date_posted DATE,
    fridge CHAR(1),
    washer CHAR(1),
    air_condition CHAR(1),
    wifi CHAR(1)
)
""")
conn.commit()
print("Tạo bảng houses thành công!")


Tạo bảng houses thành công!


In [22]:


for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO houses (location, price, area, date_posted, fridge, washer, air_condition, wifi)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['Location'],
        row['Price'],
        row['Area'],
        row['Date'],
        row['fridge'],
        row['washer'],
        row['air condition'],
        row['wifi']
    ))

conn.commit()
print("Đã chèn toàn bộ dữ liệu vào bảng houses!")





Đã chèn toàn bộ dữ liệu vào bảng houses!


In [15]:
# Lấy 10 dòng đầu tiên
query = "SELECT * FROM houses ;"
df = pd.read_sql(query, conn)
df




C:\Users\PC THINH\AppData\Local\Temp\ipykernel_19172\1084646440.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,id,location,price,area,date_posted,fridge,washer,air_condition,wifi
0,1,"Cẩm Lệ, Đà Nẵng",6900000.0,50.0,2025-09-07,1,1,1,1
1,2,"Hải Châu, Đà Nẵng",5000000.0,35.0,2025-09-07,1,1,1,1
2,3,"Hải Châu, Đà Nẵng",2500000.0,23.0,2025-07-07,0,0,0,1
3,4,"Hải Châu, Đà Nẵng",1000000.0,20.0,2025-07-07,0,0,0,0
4,5,"Sơn Trà, Đà Nẵng",3000000.0,5.0,2025-07-07,1,1,1,1
...,...,...,...,...,...,...,...,...,...
667,668,"Hải Châu, Đà Nẵng",1300000.0,25.0,2021-10-16,1,1,1,1
668,669,"Hải Châu, Đà Nẵng",2000000.0,25.0,2021-10-16,0,0,0,0
669,670,"Ngũ Hành Sơn, Đà Nẵng",2300000.0,25.0,2021-10-16,0,0,0,0
670,671,"Hải Châu, Đà Nẵng",1700000.0,28.0,2021-10-16,1,1,1,1


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
import pandas as pd
import mysql.connector

# Kết nối MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Minhthinh@2k5",
    database="DB_STORAGE2"
)

# Lấy toàn bộ dữ liệu từ bảng
query = "SELECT * FROM houses"
df = pd.read_sql(query, conn)

# Đóng kết nối
conn.close()

# Kiểm tra dữ liệu
print(df.shape)     # Số dòng, số cột
print(df.info())    # Kiểu dữ liệu
print(df.head())    # 5 dòng đầu tiên


(1375, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1375 entries, 0 to 1374
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1375 non-null   int64 
 1   location       1375 non-null   object
 2   price          1375 non-null   object
 3   area           1375 non-null   object
 4   date_posted    1375 non-null   object
 5   fridge         1375 non-null   object
 6   washer         1375 non-null   object
 7   air_condition  1375 non-null   object
 8   wifi           1375 non-null   object
dtypes: int64(1), object(8)
memory usage: 96.8+ KB
None
   id           location      price  area date_posted fridge washer  \
0   1    Cẩm Lệ, Đà Nẵng  6900000.0  50.0  2025-09-16      1      1   
1   2  Hải Châu, Đà Nẵng  5000000.0  35.0  2025-08-16      1      1   
2   3  Hải Châu, Đà Nẵng  2500000.0  23.0  2025-07-16      0      0   
3   4  Hải Châu, Đà Nẵng  1000000.0  20.0  2025-07-16      0      0   


C:\Users\PC THINH\AppData\Local\Temp\ipykernel_6560\324688181.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [2]:
df.describe(include="all")


,id,location,price,area,date_posted,fridge,washer,air_condition,wifi
count,1375.000000,1375,1375,1375,1375,1375,1375,1375,1375
unique,NaN,7,39,34,37,2,2,2,2
top,NaN,"Hải Châu, Đà Nẵng",1500000.0,20.0,2021-10-21,0,0,0,0
freq,NaN,334,200,356,486,973,940,961,813
mean,697.170909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,398.361852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,354.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,698.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1041.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.isnull().sum().sort_values(ascending=False)


id               0
location         0
price            0
area             0
date_posted      0
fridge           0
washer           0
air_condition    0
wifi             0
dtype: int64

In [5]:
df["price"] = pd.to_numeric(df["price"], errors="coerce")
df["area"] = pd.to_numeric(df["area"], errors="coerce")



In [6]:
df[df["price"] <= 0]


,id,location,price,area,date_posted,fridge,washer,air_condition,wifi


In [7]:
df[df.duplicated(subset=["location", "price", "area"], keep=False)]


,id,location,price,area,date_posted,fridge,washer,air_condition,wifi
0,1,"Cẩm Lệ, Đà Nẵng",6900000.0,50.0,2025-09-16,1,1,1,1
1,2,"Hải Châu, Đà Nẵng",5000000.0,35.0,2025-08-16,1,1,1,1
2,3,"Hải Châu, Đà Nẵng",2500000.0,23.0,2025-07-16,0,0,0,1
3,4,"Hải Châu, Đà Nẵng",1000000.0,20.0,2025-07-16,0,0,0,0
4,5,"Sơn Trà, Đà Nẵng",3000000.0,5.0,2025-07-16,1,1,1,1
...,...,...,...,...,...,...,...,...,...
1370,1381,"Hải Châu, Đà Nẵng",1300000.0,25.0,2021-10-23,1,1,1,1
1371,1382,"Hải Châu, Đà Nẵng",2000000.0,25.0,2021-10-23,0,0,0,0
1372,1383,"Ngũ Hành Sơn, Đà Nẵng",2300000.0,25.0,2021-10-23,0,0,0,0
1373,1384,"Hải Châu, Đà Nẵng",1700000.0,28.0,2021-10-23,1,1,1,1


In [8]:
df = df[df["price"] > 0]
df = df.drop_duplicates()
df = df.dropna(subset=["price", "area"])


In [9]:
print(df.info())
print(df.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1375 entries, 0 to 1374
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1375 non-null   int64  
 1   location       1375 non-null   object 
 2   price          1375 non-null   float64
 3   area           1375 non-null   float64
 4   date_posted    1375 non-null   object 
 5   fridge         1375 non-null   object 
 6   washer         1375 non-null   object 
 7   air_condition  1375 non-null   object 
 8   wifi           1375 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 96.8+ KB
None
                id         price         area
count  1375.000000  1.375000e+03  1375.000000
mean    697.170909  7.069607e+07    28.966545
std     398.361852  6.782813e+08    33.982833
min       1.000000  1.000000e+06     5.000000
25%     354.500000  1.500000e+06    20.000000
50%     698.000000  2.000000e+06    20.000000
75%    1041.500000  

In [ ]:
#Ép kiểu dữ liệu tiện nghi về dạng số
binary_cols = ["fridge", "washer", "air_condition", "wifi"]
for col in binary_cols:
    df[col] = df[col].astype(int)


In [11]:
df["date_posted"] = pd.to_datetime(df["date_posted"])


In [12]:
df = df[(df["price"] >= 500_000) & (df["price"] <= 10_000_000)]
df = df[(df["area"] >= 5) & (df["area"] <= 100)]


In [13]:
print(df.describe())
print(df.shape)


                id         price         area                    date_posted  \
count  1275.000000  1.275000e+03  1275.000000                           1275   
mean    699.265882  2.096157e+06    23.690196  2022-05-14 08:21:27.529411584   
min       1.000000  1.000000e+06     5.000000            2021-10-16 00:00:00   
25%     346.500000  1.500000e+06    20.000000            2021-10-21 00:00:00   
50%     692.000000  2.000000e+06    20.000000            2021-10-21 00:00:00   
75%    1038.500000  2.500000e+06    25.000000            2022-10-16 00:00:00   
max    1385.000000  6.900000e+06   100.000000            2025-10-20 00:00:00   
std     399.693685  8.501506e+05     8.553857                            NaN   

            fridge       washer  air_condition         wifi  
count  1275.000000  1275.000000    1275.000000  1275.000000  
mean      0.312941     0.338824       0.322353     0.438431  
min       0.000000     0.000000       0.000000     0.000000  
25%       0.000000     0.000000

In [14]:
df.to_csv("houses1.csv", index=False)
print("✅ Đã xuất CSV thành công!")

✅ Đã xuất CSV thành công!
